In [71]:
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from solana.publickey import PublicKey
from solana.rpc.api import Client
from dataclasses import dataclass
import pandas as pd
pd.set_option('display.max_colwidth', None)


### To-do
1. Fetch the tx fee
2. Use token prices to calculate successful trades profit
3. Fix Serum token transfers

In [75]:
ACCOUNT_TO_EXAMINE = PublicKey("XuErbiqKKqpvN2X8qjkBNo2BwNvQp1WZKZTDgxKB95r")
RPC_ENDPOINT = "http://185.209.177.4:8899"
DRIVER_PATH = "C:/Users/Amerf/Documents/chromedriver.exe"
NUM_TX_TO_FETCH = 1000

In [76]:
chrome_options = Options()
chrome_options.add_argument("--headless")
# chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])
driver = webdriver.Chrome(executable_path=DRIVER_PATH, options=chrome_options)
solanaClient = Client(RPC_ENDPOINT)

C:\Users\Amerf\AppData\Local\Temp\ipykernel_17956\2923442873.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=DRIVER_PATH, options=chrome_options)


In [77]:
@dataclass(init=False)
class TransactionDetails:
  amms: str = None
  tokenNames: str = None
  tokenAmounts: str = None
  slot: str = None
  signature: str = None
  txStatus: str = None
  blockTime: str = None

  def __str__(self):
    return (
        """
      --------------
      Signature: {}
      Status: {}
      Blocktime: {}
      Slot: {}
      AMMs: {}
      Tokens: {}
      Token Amounts: {}
      ---------------
      """
    ).format(self.signature,
             "Succeeded" if self.txStatus else "Failed",
             self.blockTime,
             self.slot,
             self.amms,
             self.tokenNames,
             self.tokenAmounts)

In [78]:
def listToStr(l):
  return ','.join(l)    


def getTokenTransfers():
  """
  Returns the token names and their amounts invovled in the swaps e.g.
    tokenNames: ["SOL", "USDC", "SOL"]
    tokenAmounts: [0.1, 0.5234, 0.1002]
  """
  allTokenTransfers = driver.find_elements(by=By.CLASS_NAME, value='action-row')
  tokenNames = []
  tokenAmounts = []
  previousTransfer = None
  for transfer in allTokenTransfers:
    currentTransfer = transfer.find_element(by=By.TAG_NAME, value='b').text

    if (currentTransfer != previousTransfer):
      tokenAmount, tokenName = currentTransfer.split()
      if (tokenAmount == "0"):
        continue
      tokenAmounts.append(tokenAmount)
      tokenNames.append(tokenName)

    previousTransfer = currentTransfer
  
  return listToStr(tokenNames), listToStr(tokenAmounts)


def getAmms():
  """
  Get the list of AMMs involved in the transaction e.g. ["Orca", "Whirpool"]
  """
  ixDetailsSection = driver.find_elements(By.CLASS_NAME, 'box-more-content')[0]
  ixDetailsBtn = ixDetailsSection.find_element(By.TAG_NAME, 'button')
  if (ixDetailsBtn.text == "Details"):
    ixDetailsBtn.click()

  amms = []
  allProgramInteractions = driver.find_elements(by=By.CLASS_NAME, value='det')
  for interaction in allProgramInteractions:
    programName = interaction.find_element(by=By.CLASS_NAME, value='ant-row').text.lower()
    if "raydium" in programName:
      amms.append("Raydium")
    elif "whirlpool" in programName:
      amms.append("Whirpool")
    elif "orca" in programName:
      amms.append("Orca")
    elif "mercurial" in programName:
      amms.append("Mercurial")
    elif "saber" in programName:
      amms.append("Saber")
    elif "tulip" in programName:
      amms.append("Tulip")
    elif "serum" in programName:
      amms.append("Serum")
    elif "lifinity" in programName:
      amms.append("Lifinity")
    elif "srmqPvymJe" in programName:
      amms.append("New Serum")
  
  return listToStr(amms)

### Test Single Transaction

In [53]:
txSig = "31yzUhDSXi6jFunsypfZir49KVBg9pxDxgjexguXrg7RBBfp2mwd4ACAK8jUHxeGNBbHqQajif7kQqBEpPhSxPbU"
txDetails = TransactionDetails()

txUrl = "http://solscan.io/tx/{}".format(txSig)
print("Transaction Link: {}".format(txUrl))
driver.get(txUrl)

try:
  WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CLASS_NAME, 'card-container')))
  txDetails.amms = getAmms()
  txDetails.tokenNames, txDetails.tokenAmounts = getTokenTransfers()
  print(txDetails)

except TimeoutException:
  print("Page could not be loaded!")

Transaction Link: http://solscan.io/tx/31yzUhDSXi6jFunsypfZir49KVBg9pxDxgjexguXrg7RBBfp2mwd4ACAK8jUHxeGNBbHqQajif7kQqBEpPhSxPbU

      --------------
      Signature: None
      Status: Failed
      Blocktime: None
      Slot: None
      AMMs: Raydium,Serum,Serum
      Tokens: USDC,LSTAR,LSTAR,USDC
      Token Amounts: 8.9363,6,433.95,6,433,7.7165
      ---------------
      


### Test Account Transactions

In [80]:
allTxDetails = []

lastNTransactions = solanaClient.get_signatures_for_address(ACCOUNT_TO_EXAMINE, limit=NUM_TX_TO_FETCH).value
for tx in lastNTransactions:
  txDetails = TransactionDetails()

  txDetails.signature = tx.signature
  txDetails.slot = tx.slot
  txDetails.blockTime = tx.block_time
  txDetails.txStatus = False if tx.err else True

  txUrl = "http://solscan.io/tx/{}".format(txDetails.signature)
  # print("Transaction Link: {}".format(txUrl))
  driver.get(txUrl)

  try:
    WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CLASS_NAME, 'card-container')))
    txDetails.amms = getAmms()
    txDetails.tokenNames, txDetails.tokenAmounts = getTokenTransfers()
    
    allTxDetails.append(txDetails)

  except TimeoutException:
    print("Page could not be loaded!")
    continue

Page could not be loaded!
Page could not be loaded!
Page could not be loaded!
Page could not be loaded!
Page could not be loaded!
Page could not be loaded!
Page could not be loaded!
Page could not be loaded!
Page could not be loaded!
Page could not be loaded!
Page could not be loaded!
Page could not be loaded!
Page could not be loaded!
Page could not be loaded!
Page could not be loaded!


In [81]:
df = pd.DataFrame(allTxDetails)
df

,amms,tokenNames,tokenAmounts,slot,signature,txStatus,blockTime
0,Raydium,"USDC,mSOL,USDC,USDC","2,501,198.66,0.010854,0.075824",162287881,5VnWrVuYrkcFeCC9hvkXrdFockTQYCMmVwdV2CBdBAQhz5AARUjDq8aLvv33MGoguPyJVdksniXtjHSAqFKw7mfo,False,1669018515
1,Raydium,"USDC,mSOL,USDC,USDC","2,501,198.66,0.010854,0.075824",162287881,zxhfmYFeAeoszDJeMPwWcPbBanYfnJm7pq6eEvCYcXkzVNQypJ2aSHE2tczTr6LjXEMwAX33e3sFwXjuhmttBqT,False,1669018515
2,Raydium,"USDC,mSOL,USDC,USDC","2,501,198.66,0.010854,0.075824",162287880,5yLCARf2o4qTT9fmxav4HVvWgXJ2kA4q8J8dsotpH7dfU2Jh6YBVQc7TuXQmmFgDGsp8QTepvWj9WjXTrH1GHGc7,False,1669018515
3,Raydium,"USDC,mSOL,USDC,USDC","2,492,197.86,0.00649,0.075824",162287867,4N6ckUMF34GMrNCw5UFDpMPk5yEKZS8kPmyJ4GR97KfqViTeJAMfLKtL4N9yknopoXcRkJtvgmurpwic6yjy2azJ,False,1669018509
4,Raydium,"USDC,mSOL,USDC,USDC","2,492,197.86,0.00649,0.075824",162287867,3YTPf9LwcMUtwo9Wv1vPejLQTBH3VLXsVmEzcSe77iGZWe3RrT5RZvuwH7eR8itjdizeM1oroDhABeuDSj44S4jZ,False,1669018509
...,...,...,...,...,...,...,...
980,Raydium,"USDC,SOL,USDC,USDC","186,15.88,185.83,0.0851",162286286,7AkarCHeZcWgE8sMNQ5zfQEJCLwEyZPp6sPVoVbeXoNw9V4AVRiMGoPcfBUP2ExzYKmS8t3pB8s96nTEGdPqen8,False,1669017792
981,Raydium,"USDC,SOL,USDC,USDC","1,272,108.59,1,270.1,0.001199",162286283,5AxsPqEtBrY8FvmpWSb3XWJkGSY1BZz6Bp3j3oCnujhDowg39wvQWKEfw6Vdpo3YMyFyv2nXoccaaXK21oBomSqV,False,1669017791
982,Raydium,"USDC,SOL,USDC,USDC","186,15.88,185.78,0.001199",162286282,2mg4PRpmAqRN6SUM775wVLfNMpULoBhaVakQSix5PYte78mRVcMgAAV7QSYqSZnXp7HWG5q74q5jkR9vUwaTaNja,False,1669017790
983,Raydium,"SOL,USDC,USDC,USDC,SOL","110.72,1,295,0.001199,1,295,110.56",162286281,2QRi3LscoP2HcBV2tzuVbG7xKYd9nCRQffkj9SXMKRbxKfQbxk7fNGVqUUr4U2goaNZzZRuKTpjfJ2hyKnZamTYj,False,1669017790


In [82]:
# 1. Successful arbitrages
df[df['txStatus'] == True]["signature"]

47     5LWbq6SKgPQeme6wk7EwZh2hkyuHzN9wXHZN2uefKKFqgcVpoNAm6y9vuCjRAtczArcLbivbQmLxFm4E1iCz93X4
63      xHP1xN72HaGPLihVZBCAKLgUcFhJ1A9D1rD7LnV9RAc7WQNxNJQQkn3Kd6K4QDUAMPvoSmL4iH1yf4BmaZmMLUU
78     5GsV9pqxNKxVQpcLHAw1FT1cFUT9K9v9SUb5qpQ8nPdCa3p3N12xizqLyKRzgPJjhE3q64QF6VZc48qu9zmZvw2j
79     5BVfMs7FsPFULWikFc2FfUkBWfmsRLmQspxxKmTRUYs2aP9aon1F2LJYi5SX8YTQW7JYZQ5BWPf8CH27DZcZEx71
82     4N9gBRuLjmmthN5PDqX71DRBm1mDR6cW2eL7VNp2NEduor2dBcgQqfVvQooSfjKYGbSXv7NGu9AK6HxotbYtw1qn
96     5YGnv8WcWxBQPiN4SLHxjEKhudaq3nn2eSoLGcyGXFcYDPCDge9qDcqDUPb24NEDtJRjqok3pgMu4KHRN35MYtQN
97     5Wv3jLmJzCAHWSXZD7gZQxSofenHsQ9DubyA4jPvcF1QzJbR3sPCJFGMSTbmsvaXbxg2TDYFBLxmrjmn6Xe4iY5K
118     DHXkjTixjZ1ecqCudnE7VQ8ud7o2C8GfffxV5ht6RvjBsqTxm7SfNQH2NAkMQHLWrzDzSks14EppAruQUgdvRBw
170    2gXbdwhMXpwUAiyGHLG8bkYGwx3oUKL6ci9L3j1TbnzBdCF8idUhswsgL1ZcK1ojnE5JUbnE5TMpq2tvtFkaoU62
234    5kaFEUWhnDPabwKLT988p5ABkhdo67bBCyKuVFKLeiei9z8sdvaP9uhXSCixn76Zcsw8uMDCfTAMyJ76pYBLQVkK
267    61wW6UR14nXrNdW4MrHH2U4FfPtEVJaPu